In [20]:
def lowercase_strings(string_list):#리스트 내용물 전부 소문자로
    return [s.lower() for s in string_list]

In [21]:
places = ["Home", 'Cafe', 'Bakery', 'Park', 'Convenience store', 'Bar']#장소 리스트
places = lowercase_strings(places)
#각 장소에서 할 수 있는 행동들. 인게임에서 구현된 상호작용들과 같아야 한다
house_activities = [
    "have a meal",
    "Watch TV",
    "Read a book",
    "Exercise",
    "Sleep",
    "Clean the house",
    "Call a friend",
    "Take a shower",
    "Use a desktop",
    "paint on a tablet",
    "write a book"
]
house_activities=lowercase_strings(house_activities)
cafe_activities = [
    "Order coffee",
    "Chat with barista",
    "Read a magazine",
    "Work on laptop",
    "Meet a friend",
    "Listen to music",
    "People watch",
    "Sketch in a notebook",
]
cafe_activities=lowercase_strings(cafe_activities)
bakery_activities = [
    "Buy bread",
    "Purchase pastries",
    "Order a cake",
    "Chat with baker",
    "Watch the baking process",
    "Learn a baking tip",
    "Order coffee",
    "Relax in seating area",
]
bakery_activities=lowercase_strings(bakery_activities)
park_activities = [
    "Walk the dog",
    "Have a picnic",
    "Jog",
    "Read a book on a bench",
    "Feed the ducks",
    "Take photos",
    "Ride a bike",
    "Meditate",
]
park_activities=lowercase_strings(park_activities)
convenience_store_activities = [
    "Buy snacks",
    "Purchase drinks",
    "Get lottery tickets",
    "Chat with cashier",
    "Pick up a magazine",
    "Use the ATM",
    "Get a hot dog",
    "Buy household items",
    "Refill phone credit",
    "Get ice cream"
]
convenience_store_activities=lowercase_strings(convenience_store_activities)
bar_activities = [
    "Order a drink",
    "Play pool",
    "Listen to live music",
    "Watch a sports game",
    "Chat with bartender",
    "Dance",
    "Meet new people",
    "Sing karaoke",
    "Try a new cocktail",
    "Participate in trivia night"
]
bar_activities=lowercase_strings(bar_activities)


In [22]:
persona_ryan = '''
 name : Ryan Park
 age : 29
 Innate tendency : imaginative, patient, kind
 Learned tendency : Tamara Taylor is a children's book author who loves to create stories that capture the imaginations of young readers. She is very patient and kind when it comes to her work.
 Currently : Ryan Park is working on a project to create a new mobile app. He is also reading up on the latest technologies to stay ahead of the curve.
 Lifestyle : Ryan Park goes to bed around 1am, awakes up around 9am, eats dinner around 5pm.
'''
persona_tamara = '''
 name : Tamara Taylor
 age : 30
 Innate tendency : analytical, pragmatic, driven
 Learned tendency : Ryan Park is a software engineer who loves to solve problems. He is constantly looking for ways to improve existing systems.
 Currently : Tamara Taylor lives with a housemate, Carmen Ortiz, and is working on a new series of children's books. She is also working on a comic book series for adults.
 Lifestyle : Tamara Taylor goes to bed around 10pm, awakes up around 6am, eats dinner around 6pm.
'''
persona_abigail = '''
 name : Abigail Chen
 age : 25
 Innate tendency : open-minded, curious, determined
 Learned tendency : Abigail Chen is a digital artist and animator who loves to explore how technology can be used to express ideas. She is always looking for new ways to combine art and technology.
 Currently : Abigail Chen is working on an animation project for a client. She is also experimenting with different tools and techniques to create interactive art.
 Lifestyle : Abigail Chen awakes up around 08:00, goes to bed around 24:00.
'''

In [23]:
sysprom = "You are an imaginative game story writer. Answer the request."

In [24]:
oldScedule = "'7:00, home, sleep'"#인물들의 행동을 정리할 변수

In [25]:
import ollama

def setUserProm(time,persona):
    userprom = f"""Understand the persona of the given people, {persona}.
    Around the people, there are places such as {places}.
    What you can do at home is {house_activities}. 
    What you can do at a cafe is {cafe_activities}. 
    What you can do at a bakery is {bakery_activities}. 
    What you can do at a park is {park_activities}. 
    What you can do at a convenience store is {convenience_store_activities}. 
    What you can do at a bar is {bar_activities}.
    This is what a person has done before. {oldScedule}  
    people will have coffee with her friend at the cafe at 3 p.m.
    Make one prediction about what a person will do at {time}. Make a prediction based on the given persona and what you can do in the surrounding places and places. Answer in one concise sentence, no more than 500 characters.
    """
    return userprom

In [26]:
def convert_time(time_str):
    # 시간과 분을 분리합니다.
    hours, minutes = map(int, time_str.split(':'))
    
    # 오전/오후를 결정합니다.
    period = 'am' if hours < 12 else 'pm'
    
    # 12시간 형식으로 시간을 변환합니다.
    hours = hours if 1 <= hours <= 12 else abs(hours - 12)
    
    # 결과 문자열을 반환합니다.
    return f'{hours}:{minutes:02d} {period}'


In [27]:
#챗봇에게 다음 스케줄 만들어달라고 하는 함수
def make_schedule(time,persona):
    time = convert_time(time)
    stream = ollama.chat(
    model='llama3',
    messages=[
        {"role": "system", "content": sysprom},
        {"role": "user", "content": setUserProm(time,persona)}
    ],
    options={
            #  "temperature":1.2,
             "repeat_penalty": 1.3,
             "num_ctx": 1024,
            # ,"top_k":60,
            # ,"top_p":0.3
             },
    stream=True,
    )
    whole_string = ""
    for chunk in stream:
        whole_string+=chunk['message']['content']
        print(chunk['message']['content'], end='', flush=True)
    print('\n')
    return whole_string

In [28]:
def search_place(chatbot_response):
    stream = ollama.chat(
    model='llama3',
    messages=[
        {"role": "system", "content": f'Understand the given sentence and answer it succinctly by choosing one of the most appropriate places from the list. {places}'},
        {"role": "user", "content": chatbot_response}
    ],
    options={"temperature":0.5
            },
    stream=False,
    )
    print(f"\n{stream['message']['content']}\n")
    return {stream['message']['content']}

In [29]:
for i in range(4, 11):
    a = make_schedule(f'{i*2}:00',persona_ryan)
    search_place(a)
    

Ryan Park will wake up around 9am after his usual late night of working on his mobile app project and then head to the park for a morning jog or walk with his dog before starting his day.


'park'

Ryan Park will likely wake up around 9am at home to continue reading about latest technologies for his mobile app project before starting work at 10:00 am.


home

At noon, I predict Ryan Park will "use his desktop" to work on his mobile app project at home, leveraging his innate tendency for imagination and learned skillset as a creative problem-solver.


I think there might be some confusion! Since Ryan Park is a person's name, it seems unlikely that he would use his own desktop (which is typically found in an office or home setting) to work on a mobile app project at the park.

Instead, I'd suggest choosing 'home' as the most suitable place for Ryan Park to work on his project.

Based on Ryan Park's imaginative nature and current project work schedule, I predict that he will spend time at

In [18]:
make_schedule('9:00',persona_ryan)

Based on Ryan Park's innate tendency to be imaginative and patient, I predict that he will spend his morning at home using his desktop or tablet to work on a creative project related to his mobile app development, likely starting around 9:00 am.



"Based on Ryan Park's innate tendency to be imaginative and patient, I predict that he will spend his morning at home using his desktop or tablet to work on a creative project related to his mobile app development, likely starting around 9:00 am."

에러 발생시 인게임에서 알 수 있도록 에러가 난 원래 문장이 무엇인지 같이 전달하면 좋을 듯

In [30]:
def find_places_in_string(places, enter_str):
    # List to hold places found in the string
    found_places = []

    # Iterate over each place in the places list
    for place in places:
        # Check if the place is in the enter_str
        if place in enter_str:
            found_places.append(place)

    # Print the found places
    for place in found_places:
        print(place)

# Example usage
places = ["home", "cafe", "bakery", "park", "convenience store", "bar"]
enter_str = 'Based on the options provided, I would say that "home" is the most likely place where Ryan Park will take a break from his project to read a book or sketch something.'

find_places_in_string(places, enter_str)


home
